# Sequence Classification using Recurrent Neural Networks(RNN)
In this homework, you will learn how to train a recurrent neural network for human action classification. RNN is designed handle sequential data. The network can incorporate both past history and current input. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) is a very good tutorial. You should read it before you start.

In [1]:
!pip install h5py
!pip install git+https://github.com/pytorch/tnt.git@master

  Cloning https://github.com/pytorch/tnt.git (to revision master) to /tmp/pip-req-build-h98cbv97
  Running setup.py bdist_wheel for torchnet ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-d9fs5zym/wheels/17/05/ec/d05d051a225871af52bf504f5e8daf57704811b3c1850d0012
Successfully built torchnet


In [1]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


use cuda: False


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Setup
Please make sure you have h5py and torchnet installed
> pip install h5py

> pip install git+https://github.com/pytorch/tnt.git@master


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25).

For your convenience, we provide the dataloader for you.


In [0]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('/content/gdrive/My Drive/data_2', subset='train')
valSet = Dataset('/content/gdrive/My Drive/data_2', subset='val')
tstSet = Dataset('/content/gdrive/My Drive/data_2', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=2, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=1, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=1, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

## Model
Pytorch has implemented different types of recurrent layers for you. For this homework, you can use any type of RNNs as you want:
> torch.nn.RNN()

> torch.nn.LSTM()

> torch.nn.GRU()

You can check details for different types of recurrent layers here: [RNN](http://pytorch.org/docs/master/nn.html#torch.nn.RNN), [LSTM]( http://pytorch.org/docs/master/nn.html#torch.nn.LSTM), [GRU](http://pytorch.org/docs/master/nn.html#torch.nn.GRU)


### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.



In [0]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
        
        self.project_layer = nn.Linear(75, 100)
        
        self.recurrent_layer = nn.LSTM(100, 100, 1)
        
        self.classify_layer = nn.Linear(100, 10)
        
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SequenceClassify()

In [0]:
################ 2nd To Do  (5 points)##################
lr = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
criterion = torch.nn.CrossEntropyLoss()


## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [6]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)
      
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data.item())
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 1
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   
  


train Epoch: 0  , Loss: 2.3187,  Accuracy: 10.00
val Epoch: 0  , Loss: 2.3228,  Accuracy: 9.50


## Submit your results on Kaggle

### Train a better model for action recognition!
Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves better accuracy on the action recognition validation set.


### Testing the model and submit on Kaggle
Testing the model on the testing set and save the results as a .csv file. 
Please submitted the results.csv file generated by predict_on_test() to Kaggle(https://www.kaggle.com/t/934b80879bd741e6ac1967195604d4d9) to see how well your network performs on the test set. 
################ 3rd To Do  (20 points, the highest 3 entries get extra 10 points) ###############


In [0]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size()
        return x.view(N, -1)

In [0]:
class KaggleSequenceClassify(nn.Module):
    def __init__(self):
        super(KaggleSequenceClassify, self).__init__()
        self.conv_layer = nn.Sequential( 
            nn.Conv2d(1, 200, kernel_size=(3, 3), stride=1),
            nn.MaxPool2d(2, 2),
            nn.LeakyReLU(0.03),
#           nn.LeakyReLU(0.06),
            
            nn.BatchNorm2d(200),
            
#             nn.LeakyReLU(0.06),
#             nn.BatchNorm2d(200),
            
            nn.Conv2d(200, 100, 2, 1),
#             nn.MaxPool2d(4, 4),
            nn.MaxPool2d(2, 2),
            nn.LeakyReLU(0.03),
            nn.BatchNorm2d(100),
            
            Flatten(),
            
            nn.Linear(3400, 100)
        )
        
        self.recurrent_layer = nn.LSTM(100, 100, 1)
        self.classify_layer = nn.Linear(100, 10)
        
        
    def forward(self, input, h_t_1=None, c_t_1=None):
        rnn_outputs = self.conv_layer(input.view(50, 1, 15, 75))
        rnn_outputs, (hn, cn) = self.recurrent_layer(rnn_outputs.view(50, 1, 100))        
        logits = self.classify_layer(rnn_outputs[:,-1])
        
        return logits

kaggle_model = KaggleSequenceClassify()
optimizer = optim.Adam(kaggle_model.parameters(), lr = 1e-4)
criterion = nn.CrossEntropyLoss()

In [10]:
num_epochs = 50
evaluate_every_epoch = 1
for e in range(num_epochs):
    run_epoch(trLD, kaggle_model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, kaggle_model, criterion, e, False, None) 

train Epoch: 0  , Loss: 2.2761,  Accuracy: 16.35
val Epoch: 0  , Loss: 2.2547,  Accuracy: 18.75
train Epoch: 1  , Loss: 2.1608,  Accuracy: 29.70
val Epoch: 1  , Loss: 2.0866,  Accuracy: 37.25
train Epoch: 2  , Loss: 1.9305,  Accuracy: 43.90
val Epoch: 2  , Loss: 1.7783,  Accuracy: 54.75
train Epoch: 3  , Loss: 1.6169,  Accuracy: 56.95
val Epoch: 3  , Loss: 1.4861,  Accuracy: 64.00
train Epoch: 4  , Loss: 1.3561,  Accuracy: 64.60
val Epoch: 4  , Loss: 1.2370,  Accuracy: 70.75
train Epoch: 5  , Loss: 1.1939,  Accuracy: 67.50
val Epoch: 5  , Loss: 1.1111,  Accuracy: 70.75
train Epoch: 6  , Loss: 1.0826,  Accuracy: 71.15
val Epoch: 6  , Loss: 1.0268,  Accuracy: 71.75
train Epoch: 7  , Loss: 0.9909,  Accuracy: 73.30
val Epoch: 7  , Loss: 0.9409,  Accuracy: 74.50
train Epoch: 8  , Loss: 0.9237,  Accuracy: 74.65
val Epoch: 8  , Loss: 0.8981,  Accuracy: 74.50
train Epoch: 9  , Loss: 0.8706,  Accuracy: 76.65
val Epoch: 9  , Loss: 0.8620,  Accuracy: 76.50
train Epoch: 10  , Loss: 0.8328,  Accura

In [11]:
# Use your best model to generate results on test set.

# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('/content/gdrive/My Drive/results.csv','w+')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i])+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(kaggle_model, tstLD)
print(count)

1000


## Report the performance
################ 4th To Do  (5 points)##################

In this cell, you should write an explanation of what you did (network architecture, optimiziter, learning rate, epochs) and any visualizations or graphs that you make in the process of training and evaluating your network.

Report your Kaggle Performance here:
Kaggle Accuracy - 0.81400
Kaggle Rank - 22

Network Architecture

Following are the layers in the model - 
1. (Conv2d - LeakyReLU - BatchNorm2d - Maxpool2d) * 2
1. LSTM layer. 
2. The final layer is a fully connected layer.

Optimizer: I have used Adam Optimizer which I found to be converging in 50 iterations as compared to SGD which converges in more than 200 iterations

Learning Rate: I found learning rate of 1e-4 performing better than 1e-3.

Epochs On using just a simple LSTM layer with Adam optimizer maximum accuracy reached was 45% accuracy. Finally my model converges with 97% accuracy in 50 iterations.

Used CLDNN architecture (Convolutional Long Short Memory, Fully Connected Deep Neural Network) mentioned in the paper - https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43455.pdf. 
